In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine
from sklearn.preprocessing import scale
from sklearn.metrics import classification_report

In [3]:
def funcObjetivo(c,z,data):
    N,n = data.shape
    J = 0
    for i in range(N):
        J = J + np.linalg.norm(data[i,:]-z[c[i],:])**2
    return J/N

def agrupar(z,k,data):
    N,n = data.shape
    c = np.zeros(N,dtype=int)
    for i in range(N):
        distancias= np.zeros(k)
        for j in range(k):
            distancias[j] = np.linalg.norm(z[j,:]-data[i,:])
        c[i] = np.argmin(distancias)
    for i in range(k):
        if np.sum(c==i)==0: # verifica se todos os grupos tem pelo menos um vetor
            print('Grupo',i,'não tem elementos.')
            assert 0
    return c

def representantes(c,k,data):
    N,n = data.shape
    z = np.zeros((k,n))
    for i in range(k):
        onde = np.where(c==i)
        onde = np.squeeze(onde)
        Xk = data[onde,:]
        Nk = Xk.shape[0]
        z[i,:] = np.sum(Xk, axis = 0)
        z[i,:] = z[i,:]/Nk
    return z

def sorteio_representantes(k,data):
    N,n=data.shape
    while True:
        r = np.random.randint(0,N,k)
        r.sort()
        b = r[1:]
        a = r[:-1]
        c = a < b
        if np.sum(c)==len(c):
            break
    z = data[r,:]
    return z,r


def kmeans01(data, k, maxiter, epsilon, sortear = True, r = 0):
    N,n = data.shape
    J = np.zeros(maxiter)
    
    if sortear: # inicializa com z sorteado a partir dos vetores em data
        z,r = sorteio_representantes(k,data)
    else:
        z=data[r,:]

    c = agrupar(z,k,data)
    J[0] = funcObjetivo(c,z,data)
    z=representantes(c,k,data)
    
    for i in range(1,maxiter):
        c = agrupar(z,k,data)
        J[i] = funcObjetivo(c,z,data)
        print('iter=',i,'\t\tJ=',J[i])
        if J[i]==J[i-1]: #abs(J[i]-J[i-1])/J[i]<epsilon:
            J=J[0:i+1]
            print('kmeans converge em',i,'iterações e J*=',J[i],'.')
            return r,c,J
        z=representantes(c,k,data)
        
    print('kmeans atinge o número máximo de iterações',maxiter,'.')
    return r,c,J

In [4]:
data, targets = load_wine(return_X_y = True)

targets = np.array(targets)
data = np.array(data)

In [5]:
N,n=data.shape
print('Classe 0: ', len(np.where(targets==0)[0]))
print('Classe 1: ', len(np.where(targets==1)[0]))
print('Classe 2: ', len(np.where(targets==2)[0]))


print('No total são ', N, 'vinhos.')

Classe 0:  59
Classe 1:  71
Classe 2:  48
No total são  178 vinhos.


In [6]:
# normalizar = True
# X = data
# if normalizar:
#     maxdata = np.max(data[0])
#     X = X/maxdata

X = scale(data)

k = 3
maxiter = 100
epsilon = 1e-6

Jmin = 1e30
rmin = np.zeros(k)
for i in range(k):
    print('rodada=',i)
    r, c, J = kmeans01(X, k, maxiter, epsilon)
    if min(J) < Jmin:
        Jmin = np.min(J)
        rmin = r
    

r, c, J = kmeans01(X, k, maxiter, epsilon, sortear = False, r = rmin)

rodada= 0
iter= 1 		J= 7.853057093880784
iter= 2 		J= 7.333399246343021
iter= 3 		J= 7.2127743237090165
iter= 4 		J= 7.198052663175715
iter= 5 		J= 7.190821083005315
iter= 6 		J= 7.190821083005315
kmeans converge em 6 iterações e J*= 7.190821083005315 .
rodada= 1
iter= 1 		J= 9.727553486054402
iter= 2 		J= 8.119180741981058
iter= 3 		J= 7.285172228160975
iter= 4 		J= 7.184259309153685
iter= 5 		J= 7.181735012437612
iter= 6 		J= 7.179373532835073
iter= 7 		J= 7.179373532835073
kmeans converge em 7 iterações e J*= 7.179373532835073 .
rodada= 2
iter= 1 		J= 9.553488704670228
iter= 2 		J= 8.32724729524538
iter= 3 		J= 7.293230631898807
iter= 4 		J= 7.205945083810374
iter= 5 		J= 7.202098397671318
iter= 6 		J= 7.191069653713796
iter= 7 		J= 7.184576016148346
iter= 8 		J= 7.181735012437612
iter= 9 		J= 7.179373532835073
iter= 10 		J= 7.179373532835073
kmeans converge em 10 iterações e J*= 7.179373532835073 .
iter= 1 		J= 9.727553486054402
iter= 2 		J= 8.119180741981058
iter= 3 		J= 7.2851722

In [7]:
print(classification_report(targets,c))

              precision    recall  f1-score   support

           0       0.95      1.00      0.98        59
           1       0.06      0.04      0.05        71
           2       0.00      0.00      0.00        48

    accuracy                           0.35       178
   macro avg       0.34      0.35      0.34       178
weighted avg       0.34      0.35      0.34       178



In [10]:
print('Agrupamento do KMeans:')
print('Quantidade da classe 0: ', np.count_nonzero(c == 0))
print('Quantidade da classe 1: ', np.count_nonzero(c == 1))
print('Quantidade da classe 2: ', np.count_nonzero(c == 2))


Agrupamento do KMeans:
Quantidade da classe 0:  62
Quantidade da classe 1:  51
Quantidade da classe 2:  65
